## Assignment: Student Management Database in PostgreSQL
Objective
In this exercise, you will connect to a PostgreSQL database you created, build the tables for a simple student management system, insert sample data, and perform different queries — including JOIN operations.

Part 1: Database Connection
Create a new database in PostgreSQL named student_management.

Write Python code (using psycopg2) to connect to your student_management database.

Example:

In [1]:
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="student_management",
    user="postgres",
    password="1475369",
    host="localhost",
    # port="5432"
)

cur = conn.cursor()
print("Database connected successfully.")


Database connected successfully.


## Part 2: Create Tables

Create the following tables using SQL:

### `students`
- `student_id` (SERIAL PRIMARY KEY)  
- `name` (VARCHAR)  
- `email` (VARCHAR)  
- `phone` (VARCHAR)  

### `courses`
- `course_id` (SERIAL PRIMARY KEY)  
- `course_name` (VARCHAR)  
- `credits` (INT)  

### `enrollments`
- `enrollment_id` (SERIAL PRIMARY KEY)  
- `student_id` (INT REFERENCES students(student_id))  
- `course_id` (INT REFERENCES courses(course_id))  
- `grade` (VARCHAR)  


In [10]:
tables = [
    """
    CREATE TABLE IF NOT EXISTS students(
        student_id SERIAL PRIMARY KEY,
        name VARCHAR(100),
        email VARCHAR(100),
        phone VARCHAR(25)
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS courses(
        course_id SERIAL PRIMARY KEY,
        course_name VARCHAR(100),
        credits INT
    )
    """,
    """
    CREATE TABLE IF NOT EXISTS enrollments(
        enrollment_id SERIAL PRIMARY KEY,
        student_id INT REFERENCES students(student_id),
        course_id INT REFERENCES courses(course_id),
        grade VARCHAR(50)
    )
    """
]

for query in tables:
    cur.execute(query)
    conn.commit()

## Part 3: Insert Sample Data
Insert at least 3 records into each table.

Example:

In [12]:
filling_tables = [
    """INSERT INTO students (name, email, phone) VALUES
    ('Alice Johnson', 'alice@example.com', '1234567890'),
    ('Bob Smith', 'bob@example.com', '9876543210'),
    ('Charlie Brown', 'charlie@example.com', '5555555555');""",

    """INSERT INTO courses (course_name, credits) VALUES
    ('Mathematics', 3),
    ('Computer Science', 4),
    ('History', 2);""",

    """INSERT INTO enrollments (student_id, course_id, grade) VALUES
    (1, 1, 'A'),
    (1, 2, 'B'),
    (2, 3, 'A'),
    (3, 2, 'C');"""
]
for query in filling_tables:
    cur.execute(query)
    conn.commit()

## Part 4: Query Tasks

Write SQL queries to answer the following questions:

1. **List all students.**
   ```sql
   SELECT * FROM students;


2. Find students who have a grade of 'A'.
Hint: Use a WHERE clause in the enrollments table to filter by grade.

SELECT * 
FROM enrollments
WHERE grade = 'A';


In [16]:
query = "SELECT * FROM students;"
cur.execute(query)
result = cur.fetchall()
for i in result:
    print(i)

(1, 'Alice Johnson', 'alice@example.com', '1234567890')
(2, 'Bob Smith', 'bob@example.com', '9876543210')
(3, 'Charlie Brown', 'charlie@example.com', '5555555555')


3. Show all courses with their credit hours.
Hint: Select course name and credit hours from the courses table.


SELECT course_name, credit_hours 
FROM courses;

In [23]:
query = r"SELECT course_name, credits FROM courses;"
cur.execute(query)
result = cur.fetchall()
for i in result:
    print(i)

('Mathematics', 3)
('Computer Science', 4)
('History', 2)


4. Find the courses a specific student (e.g., 'Alice Johnson') is enrolled in.
Hint: Use JOIN between students, enrollments, and courses.


SELECT s.name, c.course_name
FROM students s
JOIN enrollments e ON s.student_id = e.student_id
JOIN courses c ON e.course_id = c.course_id
WHERE s.name = 'Alice Johnson';

In [25]:
query = """
    SELECT s.name, c.course_name
    FROM students s
    JOIN enrollments e ON s.student_id = e.student_id
    JOIN courses c ON e.course_id = c.course_id
    WHERE s.name = 'Alice Johnson';
"""
cur.execute(query)
result = cur.fetchall()
for i in result:
    print(i)

('Alice Johnson', 'Mathematics')
('Alice Johnson', 'Computer Science')


5. List each student along with the number of courses they are enrolled in.
Hint: Use JOIN, GROUP BY, and COUNT().


SELECT s.name, COUNT(e.course_id) AS total_courses
FROM students s
JOIN enrollments e ON s.student_id = e.student_id
GROUP BY s.name;

In [26]:
query = """
    SELECT s.name, COUNT(e.course_id) AS total_courses
    FROM students s
    JOIN enrollments e ON s.student_id = e.student_id
    GROUP BY s.name;
"""
cur.execute(query)
result = cur.fetchall()
for i in result:
    print(i)

('Alice Johnson', 2)
('Bob Smith', 1)
('Charlie Brown', 1)


6. Find students who are not enrolled in any course.
Hint: Use LEFT JOIN and check for NULL in the enrollments table.


SELECT s.name
FROM students s
LEFT JOIN enrollments e ON s.student_id = e.student_id
WHERE e.course_id IS NULL;


In [27]:
query = """
    SELECT s.name
    FROM students s
    LEFT JOIN enrollments e ON s.student_id = e.student_id
    WHERE e.course_id IS NULL;
"""
cur.execute(query)
result = cur.fetchall()
for i in result:
    print(i)

7. Show all enrollments sorted by grade (highest to lowest).
Hint: Use ORDER BY with DESC.


SELECT * 
FROM enrollments
ORDER BY grade DESC;



In [28]:
query = """
    SELECT * 
    FROM enrollments
    ORDER BY grade DESC;
"""
cur.execute(query)
result = cur.fetchall()
for i in result:
    print(i)

(4, 3, 2, 'C')
(2, 1, 2, 'B')
(1, 1, 1, 'A')
(3, 2, 3, 'A')


Part 5: Bonus
Write a query to find the average grade per course (Convert grades into numeric values if you want to calculate an average).

Add a instructor table and modify your queries to include instructor names.



In [29]:
query = """
    SELECT c.course_name, 
    avg(
        CASE e.grade
        WHEN 'A' THEN 4.00
        WHEN 'B' THEN 3.00
        WHEN 'C' THEN 2.00
        WHEN 'D' THEN 1.00
        ELSE NULL
    END) AS average_grade
    FROM courses c JOIN enrollments e ON c.course_id = e.course_id
    GROUP BY c.course_name
"""
cur.execute(query)
result = cur.fetchall()
for i in result:
    print(i)

('Computer Science', Decimal('2.5000000000000000'))
('Mathematics', Decimal('4.0000000000000000'))
('History', Decimal('4.0000000000000000'))
